In [1]:
from osgeo import gdal
from osgeo import ogr
from osgeo import osr

def import_data_gjson(path):
    gjson_ds = ogr.Open(path)
    gjson_layer = gjson_ds.GetLayer()
    return gjson_layer

fp = r"C:\Users\Matti Katajisto\OneDrive - Aalto University\GIS-4030 GIS-development\Repo\GIS-development-course\test_data\rovaniemi_buildings\rovaniemi_buildings.geojson"
out_raster = r"C:\Users\Matti Katajisto\OneDrive - Aalto University\GIS-4030 GIS-development\Repo\GIS-development-course\test_data\rovaniemi_buildings"
pixel_size = 0.0001
rasterize_attribute = "dataSource"

gjson_layer = import_data_gjson(fp)

# Get the extent and spatial reference of the input layer
layer_extent = gjson_layer.GetExtent()
layer_srs = osr.SpatialReference()
layer_srs.ImportFromEPSG(3067)
print(type(layer_extent))
print(type(layer_srs))


: 

: 

In [ ]:


# Calculate the width and height of the output raster based on the extent and pixel size
width = int((layer_extent[1] - layer_extent[0]) / pixel_size)
height = int((layer_extent[3] - layer_extent[2]) / pixel_size)

# Create a new GeoTIFF file with the desired size and extent
driver = gdal.GetDriverByName("GTiff")
output_ds = driver.Create(out_raster, width, height, 1, gdal.GDT_Byte)
output_ds.SetGeoTransform((layer_extent[0], pixel_size, 0, layer_extent[3], 0, -pixel_size))
output_ds.SetProjection(layer_srs.ExportToWkt())

# Set the rasterization options
options = gdal.RasterizeOptions(attribute=rasterize_attribute, burnValues=[1])

# Rasterize the input layer to the output raster
gdal.RasterizeLayer(output_ds, [1], geojson_layer, options=options)

# Close the files
geojson_ds = None
output_ds = None

AttributeError: 'NoneType' object has no attribute 'SetGeoTransform'

In [43]:
from osgeo import gdal
from osgeo import ogr

# Set input raster and vector  filepaths
input_raster = r"C:\Users\Matti Katajisto\OneDrive - Aalto University\GIS-4030 GIS-development\Repo\GIS-development-course\test_data\rovaniemi.tif"
input_vector = r"C:\Users\Matti Katajisto\OneDrive - Aalto University\GIS-4030 GIS-development\Repo\GIS-development-course\test_data\rovaniemi_buildings\rovaniemi_buildings.geojson"

# Open the input raster and vector files
raster_ds = gdal.Open(input_raster)
vector_ds = ogr.Open(input_vector)
vector_layer = vector_ds.GetLayer()


In [44]:

# Get the raster geotransform and projection
geotransform = raster_ds.GetGeoTransform()
projection = raster_ds.GetProjection()

# Loop over the vector features and select the corresponding pixels in the raster
i=0
for vector_feature in vector_layer:
    # Get the geometry of the vector feature
    geometry = vector_feature.GetGeometryRef()

    # Convert the geometry to pixel coordinates in the raster
    ulx, xres, xskew, uly, yskew, yres = geotransform
    px = int((geometry.GetX() - ulx) / xres)
    py = int((geometry.GetY() - uly) / yres)

    # Get the pixel value of the selected pixel
    band = raster_ds.GetRasterBand(1)
    pixel_value = band.ReadAsArray(px, py, 1, 1)[1,1]

    
    if pixel_value != None:
        new_value =  1
        band.WriteArray(new_value, px, py)
    else:
        continue
    
    i=+1
    

# Close the raster and vector files
raster_ds = None
vector_ds = None
print(i)

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


In [41]:
print(i)
#x = raster_ds.GetRasterBand(1)
#print(x.ReadAsArray())

0


In [37]:
#px, py, 1, 1
print(x.ReadAsArray(px,py,1,1))

None


In [45]:
raster_ds = None
vector_ds = None